In [1]:
import numpy as np
import pandas as pd
from groq import Groq
import os
import time

In [2]:
df = pd.read_excel('../../data/raw/OMIEC_NEUROMORPHIC_20_08_24.xlsx')

In [3]:
df['Abstract']

0      Organic electrochemical transistors are a prom...
1      Organic mixed ionic-electronic conductors (OMI...
2      Organic mixed ionic-electronic conductors (OMI...
3      The field of organic mixed ionic-electronic co...
4      Operational stability underpins the successful...
                             ...                        
110    In recent years, organic electrochemical trans...
111    Organic electrochemical transistors (OECTs) ma...
112    Owing to their potential advantages such as lo...
113    Due to similar transmission characteristics of...
114    Synaptic plasticity functions play a crucial r...
Name: Abstract, Length: 115, dtype: object

In [4]:
GROQ_API_KEY = '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'

In [5]:
# groq
client = Groq(
    api_key=GROQ_API_KEY,
)

# respostas
responses_list = []

# todos os abstracts
total = len(df['Abstract'])
total_time = 0


for index, row in df.iterrows():
    start_time = time.time()

    # tempo
    print('\n', index, 'of', total, 'remaining estimated time', (total-index)*(total_time/(index+1)))

    # chat request
    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {
                "role": "system",
                "content": "RolePlay as a bot seeking for polymers of the type OMIEC, also known as Organic Mixed Ionic Electronic Conductors."
            },
            {
                "role": "user",
                "content": "For the text I will input next, identify any Organic Mixed Ionic Electronic Conductors (OMIEC) polymers that are used for neuromorphic devices. Provide only polymers that are mentioned in the text and provide only their names, separated by semicolons. If no such polymers are found in the text, the output should be 'None'. Do not repeat any polymer name. Do not output the polymer type. Do not output other types of materials. If the polymer name is written in full, replace it with its abbreviation."

            },
            {
                "role": "assistant",
                "content": "Hello, I will tell the polymers that are used in neuromorphic devices of any text you input next"
            },
            {
                "role": "user",
                "content": f"{row['Abstract']}"  # Use f-string to format the abstract
            }
        ],
        temperature=0.3,
        max_tokens=1024,
        top_p=0.9,
        stream=True,
        stop=None,
    )

    # coletar respostas
    response = ""

    # Stream the completion result and print the content
    for chunk in completion:
        response += chunk.choices[0].delta.content or ""
        print(chunk.choices[0].delta.content or "", end="")

    print()
    
    # listas de respostas
    responses_list.append({ 'index': index, 'response': response })

    # tempo total
    total_time += time.time() - start_time
    


 0 of 115 remaining estimated time 0.0
P3T; NDI

 1 of 115 remaining estimated time 24.15889620780945
PEDOT:PSS; Mel

 2 of 115 remaining estimated time 31.533839066823326
P3MEEET

 3 of 115 remaining estimated time 33.089569091796875
None

 4 of 115 remaining estimated time 34.553208732604986
None

 5 of 115 remaining estimated time 38.002070585886635
None

 6 of 115 remaining estimated time 36.79172781535557
None

 7 of 115 remaining estimated time 35.71095013618469
None

 8 of 115 remaining estimated time 35.55737845102946
None

 9 of 115 remaining estimated time 34.9740149974823
P-3O; P-4O; P-5O; P-6O

 10 of 115 remaining estimated time 35.60716964981773
PEDOT:PSS

 11 of 115 remaining estimated time 35.65266513824463
None

 12 of 115 remaining estimated time 34.96230272146372
PBBT-Me; PBBT-H; BBL

 13 of 115 remaining estimated time 34.57738198552813
None

 14 of 115 remaining estimated time 43.601437441507976
oEG-substituted regioregular polythiophene

 15 of 115 remaining esti

In [6]:
# DataFrame
df_with_responses = pd.DataFrame(responses_list)

# salvar para .csv
df_with_responses.to_excel('../../dataframes/llama3-70B-OMIEC200824_0.3_temp_NEUROMORPHIC_RESPONSES.xlsx', index=False)

#### Playground

In [15]:
df_with_responses

,index,response
0,0,PEDOT
1,1,None
2,2,PEDOT:PolyDADMA; PEDOT:PolyDADMA FSI; PEDOT:Po...
3,3,conducting/semiconducting conjugated polymers
4,4,PEDOT:PSS
...,...,...
870,870,None
871,871,None
872,872,None
873,873,None
